# 导入环境

In [5]:
from datasets import Dataset
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer, GenerationConfig

In [6]:
# 将JSON文件转换为CSV文件
df = pd.read_json('./dataset/output_train.json')
ds = Dataset.from_pandas(df)

In [8]:
df

,instruction,input,output
0,有一个英文到法文的词汇表，包含以下对应词汇：\n\n1. the -> le\n2. cat...,"选择题 1：\n英文句子 ""the cat jumps over the moon"" 翻译成...",D
1,有一个英文到法文的词汇表，包含以下对应词汇：\n\n1. the -> le\n2. cat...,"选择题 2：\n英文句子 ""the cow plays the fiddle"" 翻译成法文是...",A
2,有一个英文到法文的词汇表，包含以下对应词汇：\n\n1. the -> le\n2. cat...,"选择题 3：\n英文句子 ""the egg falls off the wall"" 翻译成法...",A
3,"有一个方程的系数分别为A, B和C，下面是一些已知的规则：\n\n1. 如果B^2 - 4A...","选择题 1：\n当系数A=7, B=9, C=5时，方程的解是什么？ A: -1.28571...",C
4,"有一个方程的系数分别为A, B和C，下面是一些已知的规则：\n\n1. 如果B^2 - 4A...","选择题 2：\n当系数A=7, B=9, C=5时，方程是否有解为6？ A: 是; B: 否",B
5,"有一个方程的系数分别为A, B和C，下面是一些已知的规则：\n\n1. 如果B^2 - 4A...","选择题 3：\n当系数A=7, B=9, C=5时，方程是否有解为8？ A: 是; B: 否",B
6,"有一个方程的系数分别为A, B和C，下面是一些已知的规则：\n\n1. 如果B^2 - 4A...","选择题 4：\n当系数A=7, B=6, C=0时，方程是否有解为6？ A: 是; B: 否",B
7,在一场山地自行车比赛中，四位选手分别取得了第一、第二、第三和第四名。不同的颜色代表不同的排名...,选择题 1:\n根据比赛结果，排在第二名之后的可能是哪些名次？ A: 第一名 第三名; B:...,B
8,在一场山地自行车比赛中，四位选手分别取得了第一、第二、第三和第四名。不同的颜色代表不同的排名...,选择题 2:\n第一名是不是在第三名之前？ A: 是; B: 否,A
9,在一场山地自行车比赛中，四位选手分别取得了第一、第二、第三和第四名。不同的颜色代表不同的排名...,选择题 3:\n第一名是不是在第一名之前？ A: 是; B: 否,B


# 处理数据集

In [8]:
tokenizer = AutoTokenizer.from_pretrained('/share/new_models/qwen/Qwen2-7B-Instruct', use_fast=False, trust_remote_code=True)
tokenizer

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Qwen2Tokenizer(name_or_path='/share/new_models/qwen/Qwen2-7B-Instruct', vocab_size=151643, model_max_length=131072, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '<|im_end|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|im_start|>', '<|im_end|>']}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	151643: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151644: AddedToken("<|im_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151645: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [9]:
def process_func(example):
    MAX_LENGTH = 384    # Llama分词器会将一个中文字切分为多个token，因此需要放开一些最大长度，保证数据的完整性
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer(f"<|im_start|>system\n你是一个逻辑推理专家，擅长解决逻辑推理问题。以下是一个逻辑推理的题目，形式为单项选择题。所有的问题都是（close-world assumption）闭世界假设，即未观测事实都为假。<|im_end|>\n<|im_start|>user\n{example['instruction'] + example['input']}<|im_end|>\n<|im_start|>assistant\n", add_special_tokens=False)  # add_special_tokens 不在开头加 special_tokens
    response = tokenizer(f"{example['output']}", add_special_tokens=False)
    input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.pad_token_id]
    attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]  # 因为eos token咱们也是要关注的所以 补充为1
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.pad_token_id]  
    if len(input_ids) > MAX_LENGTH:  # 做一个截断
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [10]:
tokenized_id = ds.map(process_func, remove_columns=ds.column_names)
tokenized_id

Map:   0%|          | 0/1421 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 1421
})

In [11]:
tokenizer.decode(tokenized_id[0]['input_ids'])

'<|im_start|>system\n你是一个逻辑推理专家，擅长解决逻辑推理问题。以下是一个逻辑推理的题目，形式为单项选择题。所有的问题都是（close-world assumption）闭世界假设，即未观测事实都为假。<|im_end|>\n<|im_start|>user\n有一个英文到法文的词汇表，包含以下对应词汇：\n\n1. the -> le\n2. cat -> chat\n3. jumps -> sauts\n4. over -> sur\n5. moon -> lune\n6. cow -> vache\n7. plays -> jouer\n8. fiddle -> violon\n9. egg -> bougre\n10. falls -> des chutes\n11. off -> de\n12. wall -> mur\n\n根据这个词汇表，翻译以下英文句子成法文：选择题 1：\n英文句子 "the cat jumps over the moon" 翻译成法文是： A: le chat saute sur la lune; B: le chat sauts sur le lune; C: le sauts chat sur le lune; D: le chat sauts sur le lune<|im_end|>\n<|im_start|>assistant\nD<|endoftext|>'

In [12]:
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_id[1]["labels"])))

'A<|endoftext|>'

# 创建模型

In [13]:
import torch

model = AutoModelForCausalLM.from_pretrained('/share/new_models/qwen/Qwen2-7B-Instruct', device_map="auto",torch_dtype=torch.bfloat16)
model

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 3584)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=3584, out_features=3584, bias=True)
          (k_proj): Linear(in_features=3584, out_features=512, bias=True)
          (v_proj): Linear(in_features=3584, out_features=512, bias=True)
          (o_proj): Linear(in_features=3584, out_features=3584, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (up_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (down_proj): Linear(in_features=18944, out_features=3584, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm()
        (post_attention_layernorm): Qwen2RMSNorm()
      )
    )
    (norm): Qwen2RMSNorm()
  )
  (lm_head): Linear(

In [15]:
model.enable_input_require_grads() # 开启梯度检查点时，要执行该方法

In [16]:
model.dtype

torch.bfloat16

# lora 

In [17]:
from peft import LoraConfig, TaskType, get_peft_model

config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, 
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    inference_mode=False, # 训练模式
    r=8, # Lora 秩
    lora_alpha=32, # Lora alaph，具体作用参见 Lora 原理
    lora_dropout=0.1# Dropout 比例
)
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'gate_proj', 'k_proj', 'down_proj', 'v_proj', 'up_proj', 'q_proj', 'o_proj'}, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)

In [18]:
model = get_peft_model(model, config)
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='/share/new_models/qwen/Qwen2-7B-Instruct', revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'gate_proj', 'k_proj', 'down_proj', 'v_proj', 'up_proj', 'q_proj', 'o_proj'}, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)

In [19]:
model.print_trainable_parameters()

trainable params: 20,185,088 || all params: 7,635,801,600 || trainable%: 0.2643


# 配置训练参数

In [20]:
args = TrainingArguments(
    output_dir="./output/Qwen2_instruct_lora",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    logging_steps=10,
    num_train_epochs=3,
    save_steps=100, # 为了快速演示，这里设置10，建议你设置成100
    learning_rate=1e-4,
    save_on_each_node=True,
    gradient_checkpointing=True
)

In [21]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_id,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

/root/.conda/envs/xtuner0121/lib/python3.10/site-packages/accelerate/accelerator.py:451: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [22]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss
10,3.743900
20,0.345300
30,0.325400
40,0.398900
50,0.343000
60,0.281000
70,0.276400
80,0.276600
90,0.321300
100,0.152200


/root/.conda/envs/xtuner0121/lib/python3.10/site-packages/peft/utils/save_and_load.py:195: UserWarning: Could not find a config file in /share/new_models/qwen/Qwen2-7B-Instruct - will assume that the vocabulary was not modified.
  warnings.warn(
/root/.conda/envs/xtuner0121/lib/python3.10/site-packages/peft/utils/save_and_load.py:195: UserWarning: Could not find a config file in /share/new_models/qwen/Qwen2-7B-Instruct - will assume that the vocabulary was not modified.
  warnings.warn(


TrainOutput(global_step=267, training_loss=0.29274509367201657, metrics={'train_runtime': 1693.8551, 'train_samples_per_second': 2.517, 'train_steps_per_second': 0.158, 'total_flos': 5.612190018707866e+16, 'train_loss': 0.29274509367201657, 'epoch': 3.0})

# 合并加载模型

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from peft import PeftModel

mode_path = '/share/new_models/qwen/Qwen2-7B-Instruct/'
lora_path = './output/Qwen2_instruct_lora/checkpoint-200' # 这里改称你的 lora 输出对应 checkpoint 地址

# 加载tokenizer
tokenizer = AutoTokenizer.from_pretrained(mode_path, trust_remote_code=True)

# 加载模型
model = AutoModelForCausalLM.from_pretrained(mode_path, device_map="auto",torch_dtype=torch.bfloat16, trust_remote_code=True).eval()

# 加载lora权重
model = PeftModel.from_pretrained(model, model_id=lora_path)
model = model.merge_and_unload() 
model.save_pretrained("./merge")

tokenizer.save_pretrained("./merge")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

('./merge/tokenizer_config.json',
 './merge/special_tokens_map.json',
 './merge/vocab.json',
 './merge/merges.txt',
 './merge/added_tokens.json',
 './merge/tokenizer.json')

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from peft import PeftModel

# mode_path = '/share/new_models/qwen/Qwen2-7B-Instruct/'
# lora_path = './output/Qwen2_instruct_lora/checkpoint-200' # 这里改称你的 lora 输出对应 checkpoint 地址
mode_path = './merge/'
# 加载tokenizer
tokenizer = AutoTokenizer.from_pretrained(mode_path, trust_remote_code=True)

# 加载模型
model = AutoModelForCausalLM.from_pretrained(mode_path, device_map="auto",torch_dtype=torch.bfloat16, trust_remote_code=True).eval()

# 加载lora权重
# model = PeftModel.from_pretrained(model, model_id=lora_path)

prompt = "题目如下：有一个列表，找出该列表的最后一个元素。\n\n下列选项中哪个是列表 `[a, b, c, d]` 的最后一个元素？A: a; B: b; C: c; D: d"
       
inputs = tokenizer.apply_chat_template([{"role": "user", "content": "你是一个逻辑推理专家，擅长解决逻辑推理问题。以下是一个逻辑推理的题目，形式为单项选择题。所有的问题都是（close-world assumption）闭世界假设，即未观测事实都为假。请逐步分析问题并在最后一行输出答案，最后一行的格式为`答案是：A`。"},{"role": "user", "content": prompt}],
                                       add_generation_prompt=True,
                                       tokenize=True,
                                       return_tensors="pt",
                                       return_dict=True
                                       ).to('cuda')


gen_kwargs = {"max_length": 2500, "do_sample": True, "top_k": 1}
with torch.no_grad():
    outputs = model.generate(**inputs, **gen_kwargs)
    outputs = outputs[:, inputs['input_ids'].shape[1]:]
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some parameters are on the meta device device because they were offloaded to the cpu.


D
